### Importing libraries

In [2]:
import pandas as pd
from pulp import *

### Make datafram

In [3]:
data = pd.read_excel('./Delivery truck trip data.xlsx')

/tmp/ipykernel_10280/3630388012.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data = pd.read_excel('./Delivery truck trip data.xlsx')


### Distance matrix

In [4]:
warehouse_str = data["Origin_Location"].unique()
CustomerPoint_str = data["Destination_Location"].unique()
# sorting
warehouse_str.sort()
CustomerPoint_str.sort()
# distance matrix
costs = []

for w in warehouse_str:
    distance_row = []
    for c in CustomerPoint_str:
        try:
            distance = data[(data['Origin_Location'] == w) & (data['Destination_Location'] == c)]['TRANSPORTATION_DISTANCE_IN_KM'].tolist()[0]
        except:
            distance = 10000000000000
        distance_row.append(distance)
    costs.append(distance_row)

### Get supply and demand

In [5]:
supply_str = dict(data["Origin_Location"].value_counts().sort_index())
demand_str = dict(data["Destination_Location"].value_counts().sort_index())

In [6]:
supply = {}
demand = {}
warehouse = []
CustomerPoint = []

i = 0
for s in supply_str:
    supply[str(i)] = supply_str[s]
    warehouse.append(str(i))
    i+=1

i = 0
for s in demand_str:
    demand[str(i)] = demand_str[s]
    CustomerPoint.append(str(i))
    i+=1

In [7]:
# The cost data is made into a dictionary
costs = makeDict([warehouse,CustomerPoint],costs,0)

### Optimize routes

In [8]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Routes Optimaization",LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in warehouse for b in CustomerPoint]

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route",(warehouse,CustomerPoint),0,None,LpInteger)

prob += lpSum([vars[w][b]*costs[int(w)][int(b)] for (w,b) in Routes]), "Sum_of_Transporting_Costs"

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in warehouse:
    prob += lpSum([vars[w][b] for b in CustomerPoint])<=supply[w], "Sum_of_Products_out_of_Warehouse_%s"%w

# The demand minimum constraints are added to prob for each demand node (customer)
# These constraints are stored for resolve later
customer_demand_constraint = {}
for b in CustomerPoint:
    constraint = lpSum([vars[w][b] for w in warehouse])>=demand[b]
    prob += constraint, "Sum_of_Products_into_customer_%s"%b
    customer_demand_constraint[b] = constraint


prob.writeLP("ProductDistributionProblem.lp") # The problem data is written to an .lp file

prob.solve()

/media/usama/Programming/Code files/assignment/venv/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /media/usama/Programming/Code files/assignment/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/ec5f888fbffc4c06bfe40de226da2ba8-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/ec5f888fbffc4c06bfe40de226da2ba8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 705 COLUMNS
At line 375107 RHS
At line 375808 BOUNDS
At line 469410 ENDATA
Problem MODEL has 700 rows, 93601 columns and 187200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 1.39 seconds
Cgl0004I processed model has 700 rows, 93600 columns (93600 integer (21320 of which binary)) and 187200 elements
Cbc0045I No integer variables out of 93600 objects (93600 integer) have costs
Cbc0045I branch on satisfied N create fake objective Y random cost Y
Cbc0012I Integer solution of 0 found by DiveCoeff

1

In [9]:
for v in prob.variables():
    try:
        if v.varValue > 0:
            print(warehouse_str[int(str(v.name).split('_')[1])], 'to' ,CustomerPoint_str[int(str(v.name).split('_')[2])], "=", v.varValue)
    except:
        pass

ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Pune, Pune, Maharashtra = 6.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Raipur Kutchery, Raipur, Chattisgarh = 1.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Rajkot Malviyanagar, Rajkot, Gujarat = 6.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Rampella, Jharsuguda, Odisha = 2.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Rangadhipa, Sundergarh, Odisha = 2.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Rohanda, North 24 Parganas, West Bengal = 5.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to S.F.S.Mansarovar, Jaipur, Rajasthan = 1.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Bajaj Road-Sikar, Sikar, Rajasthan = 1.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Shah Mohiuddinpur, Ghaziabad, Uttar Pradesh = 5.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Sikandarpur Sihura, Unnao, Uttar Pradesh = 3.0
ASHOK LEYLAND ENNORE,CHENNAI,TAMIL NADU to Adugodi, Bangalore, Karnataka = 1.0
Muthur, Krishnagiri, Tamil Nadu to Mahabubabad, Warangal, Telangana = 1.0
Mut